#Setup

User Variables

In [14]:
RESOLUTION = 2
SPHERE_RESOLUTION = 10
LAYERS = 3
MIN_SIGNAL = 5 #Minimum db required to communicate
DEFAULT_OPACITY = 1/(LAYERS+1)
X_MAX, Y_MAX, Z_MAX, F_MAX, db_MAX = [10,10,10, 5, 200]
FILE_LOCATION = "/content/drive/Shareddrives/Prime TTT (IL4)/DevOps/Models/Jamming Model/Sample_Jam.csv"

#Imports & Drive Mounting

In [15]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import math
import random
import ipywidgets as widgets
from IPython.display import display
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Functions

Creates a point map for a sphere to be graphed as a plotly surface

In [16]:
def plotly_sphere(x_dim, y_dim, z_dim, r_dim, resolution=SPHERE_RESOLUTION):
    u, v = np.mgrid[0:2 * np.pi:resolution * 2j, 0:np.pi:resolution * 1j]
    X_range = r_dim * np.cos(u) * np.sin(v) + x_dim
    Y_range = r_dim * np.sin(u) * np.sin(v) + y_dim
    Z_range = r_dim * np.cos(v) + z_dim
    return X_range, Y_range, Z_range

In [17]:
def on_remove_clicked(_):
  print(f"Node Removed:")
  nodes[-1].print()
  del nodes[-1]


In [18]:
def on_add_clicked(_):
  nodes.append(Node(origin_x=Xslider.value, origin_y=Yslider.value, origin_z=Zslider.value, frequency=Fslider.value, db=dbslider.value, stype=Tradio.value))
  print(f"Node Added:")
  nodes[-1].print()

In [19]:
def on_simple_graph_clicked(_):
  data = []
  for node in nodes:
    data = data + node.graph()

  fig = go.Figure(data=data)

  fig.update_traces(showscale=False)
  fig.update_layout(
    showlegend=False,
    autosize=False,
    width=1500,
    height=1000,
    paper_bgcolor="LightSteelBlue",
  )

  fig.show()

In [20]:
def on_complex_graph_clicked(_):
  data_x_jam, data_y_jam, data_z_jam, data_value_jam = [],[],[],[]
  data_x_com, data_y_com, data_z_com, data_value_com = [],[],[],[]

  node_grid = np.zeros((X_MAX * RESOLUTION,Y_MAX * RESOLUTION,Z_MAX * RESOLUTION))
  for x in range(0,X_MAX * RESOLUTION):
    for y in range(0, Y_MAX * RESOLUTION):
      for z in range(0, Z_MAX * RESOLUTION):
        for node in nodes:
          distance = node.distance_finder(x/RESOLUTION,y/RESOLUTION,z/RESOLUTION)

          signal = node.signal_strength_db(distance)
          #if signal > 0:
          #  node.print()
          #  print(f"X:{x}, Y:{y}, Z:{z}, Signal:{signal}\n")
          if node.t == 'jam' and signal > 0:
            node_grid[x,y,z] = node_grid[x,y,z] - node.signal_strength_db(distance)
          elif node.t == 'com' and signal > 0:
            node_grid[x,y,z] = node_grid[x,y,z] + node.signal_strength_db(distance)


  for x in range(0,X_MAX * RESOLUTION):
    for y in range(0, Y_MAX * RESOLUTION):
      for z in range(0, Z_MAX * RESOLUTION):
        if node_grid[x,y,z] < 0:
          #print(f"X:{x/2}, Y:{y/2}, Z:{z/2}, Signal:{node_grid[x,y,z]}")
          data_x_jam.append(x/RESOLUTION)
          data_y_jam.append(y/RESOLUTION)
          data_z_jam.append(z/RESOLUTION)
          data_value_jam.append(-node_grid[x,y,z])

        elif node_grid[x,y,z] > 0:
          data_x_com.append(x/RESOLUTION)
          data_y_com.append(y/RESOLUTION)
          data_z_com.append(z/RESOLUTION)
          data_value_com.append(node_grid[x,y,z])

  data = [go.Scatter3d(x= data_x_jam, y= data_y_jam, z= data_z_jam, mode ='markers',
                                    marker=dict(
                                      size=12,
                                      color=data_value_jam,
                                      colorscale=["rgb(255, 204, 203)", "red"]),
                                      opacity=.25,
                                      hovertemplate = 'y: %{y}'+'<br>x: %{x}'+'<br>z: %{z}<br>'+'%{text}',
                                      text = [f'Signal: {round(i,2)}' for i in data_value_jam]),
          go.Scatter3d(x= data_x_com, y= data_y_com, z= data_z_com, mode ='markers',
                                    marker=dict(
                                      size=12,
                                      color=data_value_com,
                                      colorscale=["rgb(173,216,230)","blue"]),
                                    opacity=.25,
                                    hovertemplate = 'y: %{y}'+'<br>x: %{x}'+'<br>z: %{z}<br>'+'%{text}',
                                    text = [f'Signal: {round(i,2)}' for i in data_value_com])]

  fig = go.Figure(data=data)
  fig.update_layout(
      showlegend=False,
      autosize=False,
      width=1500,
      height=1000,
      paper_bgcolor="LightSteelBlue",
  )
  fig.show()

#Node Class

**SIGNAL STRENGTH**

FSPL (dB) = 20log10(d) + 20log10(f) + K

d = distance
f = frequency
K= constant that depends on the units used for d and f
If d is measured in kilometers, f in MHz, the formula is:

FSPL (dB) = 20log10(d)+ 20log10(f) + 32.44

In [21]:
class Node:
    def __init__(self, origin_x, origin_y, origin_z, frequency, db, stype="jam", min_signal = MIN_SIGNAL):
        self.x = origin_x
        self.y = origin_y
        self.z = origin_z
        self.f = frequency*1000
        self.db = db
        self.t = stype.lower()
        self.rec_min = min_signal
        self.r = self.signal_strength_min()

    def graph(self):
      data = []
      color = "blue"
      if self.t == "jam":
          color = "red"

      for radius in np.linspace(0, self.r, LAYERS + 1):
        (X, Y, Z) = plotly_sphere(self.x, self.y, self.z, radius)
        data.append(go.Surface(x=X, y=Y, z=Z, opacity=DEFAULT_OPACITY, colorscale=[color, color], showlegend=False, showscale=False))
      return data

    def print(self):
        print(f"X: {self.x}, Y: {self.y}, Z: {self.z}, Radius: {self.r}")

    def signal_strength_min(self):
      return pow(10,(abs(self.db) - 20 * math.log10(abs(self.f)) - 32.4 - self.rec_min)/20)

    def signal_strength_db(self, distance):
      if distance == 0:
        distance = 1/RESOLUTION
      return abs(self.db) - 20 * math.log10(abs(distance)) - 20 * math.log10(abs(self.f)) - 32.4

    def distance_finder(self, x, y, z):
      return math.sqrt((self.x - x) ** 2 + (self.y - y) ** 2 + (self.z - z) ** 2)

#Widgets

Initalize the Spheres from Data

In [22]:
df = pd.read_csv(FILE_LOCATION)
nodes = []
for i, row in df.iterrows():
  nodes.append(Node(row['X'],row['Y'],row['Z'],row["frequency"],row["db"], row["Type"]))


In [23]:
Xslider = widgets.IntSlider(min=0, max=X_MAX, step=1, description='X:', value=round(X_MAX/2))
display(Xslider)

Yslider = widgets.IntSlider(min=0, max=Y_MAX, step=1, description='Y:', value=round(Y_MAX/2))
display(Yslider)

Zslider = widgets.IntSlider(min=0, max=Z_MAX, step=1, description='Z:', value=round(Z_MAX/2))
display(Zslider)

Fslider = widgets.FloatSlider(min=0, max=F_MAX, step=.1, description='Freq:', value=round(F_MAX/2))
display(Fslider)

dbslider = widgets.IntSlider(min=0, max=db_MAX, step=1, description='db:', value=round(db_MAX/2))
display(dbslider)

Tradio = widgets.RadioButtons(options=['Com', 'Jam'], value='Com', description='Type:', disabled=False)
display(Tradio)

addButton = widgets.Button(description='Add Node', disabled=False, button_style='Success', tooltip='Add a node to the graph')
addButton.on_click(on_add_clicked)
display(addButton)

removeButton = widgets.Button(description='Delete Last Node', disabled=False, button_style='Success', tooltip='Delete the last node from the graph')
removeButton.on_click(on_remove_clicked)
display(removeButton)

simple_graphButton = widgets.Button(description='Simple Graph', disabled=False, button_style='Success', tooltip='Simple Visualization')
simple_graphButton.on_click(on_simple_graph_clicked)
display(simple_graphButton)

complex_graphButton = widgets.Button(description='Complex Graph', disabled=False, button_style='Success', tooltip='Complex Visualization')
complex_graphButton.on_click(on_complex_graph_clicked)
display(complex_graphButton)

IntSlider(value=5, description='X:', max=10)

IntSlider(value=5, description='Y:', max=10)

IntSlider(value=5, description='Z:', max=10)

FloatSlider(value=2.0, description='Freq:', max=5.0)

IntSlider(value=100, description='db:', max=200)

RadioButtons(description='Type:', options=('Com', 'Jam'), value='Com')

Button(button_style='success', description='Add Node', style=ButtonStyle(), tooltip='Add a node to the graph')

Button(button_style='success', description='Delete Last Node', style=ButtonStyle(), tooltip='Delete the last n…

Button(button_style='success', description='Simple Graph', style=ButtonStyle(), tooltip='Simple Visualization'…

Button(button_style='success', description='Complex Graph', style=ButtonStyle(), tooltip='Complex Visualizatio…

#Graph

Visualization of Overlap

In [24]:
data_x_jam, data_y_jam, data_z_jam, data_value_jam = [],[],[],[]
data_x_com, data_y_com, data_z_com, data_value_com = [],[],[],[]

node_grid = np.zeros((X_MAX * RESOLUTION,Y_MAX * RESOLUTION,Z_MAX * RESOLUTION))
for x in range(0,X_MAX * RESOLUTION):
  for y in range(0, Y_MAX * RESOLUTION):
    for z in range(0, Z_MAX * RESOLUTION):
      for node in nodes:
        distance = node.distance_finder(x/RESOLUTION,y/RESOLUTION,z/RESOLUTION)

        signal = node.signal_strength_db(distance)
          #if signal > 0:
          #  node.print()
          #  print(f"X:{x}, Y:{y}, Z:{z}, Signal:{signal}\n")
        if node.t == 'jam' and signal > 0:
          node_grid[x,y,z] = node_grid[x,y,z] - node.signal_strength_db(distance)
        elif node.t == 'com' and signal > 0:
          node_grid[x,y,z] = node_grid[x,y,z] + node.signal_strength_db(distance)


In [25]:
for x in range(0,X_MAX * RESOLUTION):
  for y in range(0, Y_MAX * RESOLUTION):
    for z in range(0, Z_MAX * RESOLUTION):
      if node_grid[x,y,z] < 0:
        #print(f"X:{x/2}, Y:{y/2}, Z:{z/2}, Signal:{node_grid[x,y,z]}")
        data_x_jam.append(x/RESOLUTION)
        data_y_jam.append(y/RESOLUTION)
        data_z_jam.append(z/RESOLUTION)
        data_value_jam.append(-node_grid[x,y,z])

      elif node_grid[x,y,z] > 0:
        data_x_com.append(x/RESOLUTION)
        data_y_com.append(y/RESOLUTION)
        data_z_com.append(z/RESOLUTION)
        data_value_com.append(node_grid[x,y,z])

Visualization of Spheres

In [26]:
data = [go.Scatter3d(x= data_x_jam, y= data_y_jam, z= data_z_jam, mode ='markers',
                                    marker=dict(
                                      size=12,
                                      color=data_value_jam,
                                      colorscale=["rgb(255, 204, 203)", "red"]),
                                      opacity=.25,
                                      hovertemplate = 'y: %{y}'+'<br>x: %{x}'+'<br>z: %{z}<br>'+'%{text}',
                                      text = [f'Signal: {round(i,2)}' for i in data_value_jam]),
        go.Scatter3d(x= data_x_com, y= data_y_com, z= data_z_com, mode ='markers',
                                    marker=dict(
                                      size=12,
                                      color=data_value_com,
                                      colorscale=["rgb(173,216,230)","blue"]),
                                    opacity=.25,
                                    hovertemplate = 'y: %{y}'+'<br>x: %{x}'+'<br>z: %{z}<br>'+'%{text}',
                                    text = [f'Signal: {round(i,2)}' for i in data_value_com])]

fig = go.Figure(data=data)
fig.update_layout(
    showlegend=False,
    autosize=False,
    width=1500,
    height=1000,
    paper_bgcolor="LightSteelBlue",
)
fig.show()

# Modeling Automation

In [27]:
class Model:
  def __init__(self, signal_grid, com_freq, com_db, goal_x=X_MAX, x_grid=X_MAX, y_grid=Y_MAX, z_grid=Z_MAX, res = RESOLUTION):
    self.x = x_grid
    self.y = y_grid
    self.z = z_grid
    self.res = res
    self.f = com_freq
    self.db = com_db
    self.goal_x = goal_x
    self.signal_grid = signal_grid
    self.nodes = []
    self.reward = 0

  def place_node(self, start = 0):
    if len(self.nodes) == 0:
      y = random.randint(0, Y_MAX*self.res)/self.res
      z = random.randint(0, Z_MAX*self.res)/self.res
      self.nodes.append(Node(origin_x=0, origin_y=y, origin_z=z, frequency=self.f, db=self.db, stype="com", min_signal = MIN_SIGNAL))
      return

    else:
      (X,Y,Z) = plotly_sphere(self.nodes[-1].x,self.nodes[-1].y,self.nodes[-1].z,self.nodes[-1].r)
      i = random.randint(0, len(X) - 1)
      j = random.randint(0, len(X[i]) - 1)
      counter = 0
      while(False):#(X[i][j] > 0) or (Y[i][j] > 0) or (Y[i][j] < Y_MAX) or (Z[i][j] > 0) or (Z[i][j] < Z_MAX)):
        if counter > 1000:
          if self.nodes[-1].x > 0:
            self.nodes.append(Node(self.nodes[-1].x, random.randint(0,Y_MAX), random.randint(0,Z_MAX), self.f, self.db, stype="com", min_signal = MIN_SIGNAL))
          else:
            self.nodes.append(Node(0, random.randint(0,Y_MAX), random.randint(0,Z_MAX), self.f, self.db, stype="com", min_signal = MIN_SIGNAL))
          return
        i = random.randint(0, len(X) - 1)
        j = random.randint(0, len(X[i]) - 1)

      #LINEAR SIGNAL ADJUSTMENT - NEEDS TO BE MADE EXPONENTIAL
      signal = 0
      if self.signal_grid[round(X[i][j]),round(Y[i][j]),round(Z[i][j])] < 0:
        signal = self.signal_grid[round(X[i][j]),round(Y[i][j]),round(Z[i][j])]
      self.nodes.append(Node(X[i][j] + signal, Y[i][j], Z[i][j], self.f, self.db, stype="com", min_signal = MIN_SIGNAL))
      return

  def check_goal(self):
    return self.nodes[-1].x >= self.goal_x

  def print(self):
    for node in self.nodes:
      node.print()

  def update_grid(self):
    for x in range(round(self.nodes[-1].x - self.nodes[-1].r) * RESOLUTION,round(self.nodes[-1].x + self.nodes[-1].r) * RESOLUTION):
      for y in range(round(self.nodes[-1].y - self.nodes[-1].r) * RESOLUTION,round(self.nodes[-1].y + self.nodes[-1].r) * RESOLUTION):
        for z in range(round(self.nodes[-1].z - self.nodes[-1].r) * RESOLUTION,round(self.nodes[-1].z + self.nodes[-1].r) * RESOLUTION):
          if x >= 0 and y >= 0 and z>=0 and x<X_MAX and y<Y_MAX and z<Z_MAX:
            distance = self.nodes[-1].distance_finder(x/RESOLUTION,y/RESOLUTION,z/RESOLUTION)
            print(f"Current Signal Strength:{self.signal_grid[x,y,z]}  Updated Strength:{self.signal_grid[x,y,z] + self.nodes[-1].signal_strength_db(distance)}")
            self.signal_grid[x,y,z] = self.signal_grid[x,y,z] + self.nodes[-1].signal_strength_db(distance)

In [28]:
class Simulation:
  def __init__(self, signal_grid, com_freq, com_db, goal_x=X_MAX, x_grid=X_MAX, y_grid=Y_MAX, z_grid=Z_MAX, res = RESOLUTION):
    self.x = x_grid
    self.y = y_grid
    self.z = z_grid
    self.res = res
    self.f = com_freq
    self.db = com_db
    self.goal_x = goal_x
    self.signal_grid = signal_grid
    self.ideal_model = None

  def run(self, iterations=1000):
    for iter in range(iterations):
      print(f"Iteration {iter + 1}")

      m = Model(self.signal_grid, self.f, self.db, self.goal_x, self.x, self.y, self.z, self.res)

      m.place_node()
      m.nodes[-1].print()
      m.update_grid()

      while m.check_goal() is False:
        m.place_node()
        m.nodes[-1].print()
        m.update_grid()

      if self.ideal_model == None:
        self.ideal_model = m
      elif len(m.nodes) < len(self.ideal_model.nodes):
          self.ideal_model = m
    return self.ideal_model

In [29]:
sim = Simulation(node_grid,1.5,115)

ideal_model = sim.run(10)
print("IDEAL MODEL:")
ideal_model.print()

Streaming output truncated to the last 5000 lines.
Current Signal Strength:18.23570206083366  Updated Strength:24.09168393238084
Current Signal Strength:18.638268936372064  Updated Strength:24.872136416813248
Current Signal Strength:21.593291040975416  Updated Strength:28.118740808828726
Current Signal Strength:22.002515106035034  Updated Strength:28.712798930828477
Current Signal Strength:20.105871971125246  Updated Strength:23.029807261152186
Current Signal Strength:19.985458664575354  Updated Strength:23.440704838896977
Current Signal Strength:19.748299041765087  Updated Strength:23.7411766709386
Current Signal Strength:18.85019408156819  Updated Strength:23.37992030036947
Current Signal Strength:17.907499644122524  Updated Strength:22.963060638462103
Current Signal Strength:17.123779492014123  Updated Strength:22.68012912978687
Current Signal Strength:16.674414046458125  Updated Strength:22.68833858983762
Current Signal Strength:16.839313462410182  Updated Strength:23.2457709972664